<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/main/30_num_int/25_convergence_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This cell is for the Google Colaboratory
# https://stackoverflow.com/a/63519730
if 'google.colab' in str(get_ipython()):
  path_py = '/content/nmisp_py'

  import os
  if not os.path.exists(path_py):
    import subprocess
    subprocess.run(
        ('git', 'clone', 'https://github.com/kangwonlee/nmisp_py')
    )
  assert os.path.exists(path_py)

  import sys
  sys.path.insert(0, path_py)

In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py
import numpy as np

# 수치 적분의 수렴 차수<br>Convergence Order of Numerical Integration

수치 적분의 구간 수를 늘리면 오차가 줄어든다. 그렇다면 오차가 얼마나 빠르게 줄어드는가?<br>
As we increase the number of intervals in numerical integration, the error decreases. But how fast does the error decrease?

이 질문에 답하기 위해 **수렴 차수**라는 개념을 살펴보자.<br>
To answer this question, let's look at the concept of **convergence order**.

## 수렴 차수란?<br>What is Convergence Order?

구간의 폭 $\Delta x$와 오차 $E$ 사이에 다음 관계가 있다고 하자.<br>
Suppose there is the following relationship between the interval width $\Delta x$ and the error $E$.

$$
E \propto (\Delta x)^p
$$

여기서 $p$를 수렴 차수라 한다. 양변에 로그를 취하면:<br>
Here, $p$ is called the convergence order. Taking logarithm of both sides:

$$
\log E = p \cdot \log (\Delta x) + C
$$

따라서 양대수 (log-log) 그래프에서 오차와 구간 폭의 관계는 기울기가 $p$인 직선이 된다.<br>
Therefore, on a log-log plot, the relationship between error and interval width becomes a straight line with slope $p$.

이론적으로 알려진 수렴 차수는 다음과 같다:<br>
The theoretically known convergence orders are as follows:

| 방법<br>Method | 수렴 차수 $p$<br>Convergence Order $p$ |
|---|---|
| 0차 (직사각형)<br>0th order (Rectangle) | 1 |
| 1차 (사다리꼴)<br>1st order (Trapezoid) | 2 |
| 2차 (심슨)<br>2nd order (Simpson) | 4 |

## 적분 함수 정의<br>Define Integration Functions

앞선 노트북에서 소개한 세 가지 수치 적분 함수를 정의하자.<br>
Let's define the three numerical integration functions introduced in the previous notebooks.

In [ ]:
def get_delta_x(xi, xe, n):
    return (xe - xi) / n

### 0차 적분 (직사각형 규칙)<br>0th Order Integration (Rectangle Rule)

$$
Area = \sum_{k=0}^{n-1} f(x_k) \cdot \Delta x
$$

In [ ]:
def num_int_0(f, xi, xe, n):
    x_array = py.linspace(xi, xe, n+1)
    delta_x = x_array[1] - x_array[0]

    integration_result = 0.0
    for k in range(n):
        integration_result += f(x_array[k]) * delta_x

    return integration_result

### 1차 적분 (사다리꼴 규칙)<br>1st Order Integration (Trapezoid Rule)

$$
Area = \sum_{k=0}^{n-1} \frac{\Delta x}{2} \left[ f(x_k) + f(x_{k+1}) \right]
$$

In [ ]:
def num_int_1(f, xi, xe, n):
    x_array = py.linspace(xi, xe, n+1)
    delta_x = x_array[1] - x_array[0]

    integration_result = 0.0
    y_k = f(x_array[0])

    for k in range(n):
        y_k_plus_1 = f(x_array[k+1])
        integration_result += 0.5 * (y_k + y_k_plus_1) * delta_x
        y_k = y_k_plus_1

    return integration_result

### 2차 적분 (심슨 규칙)<br>2nd Order Integration (Simpson's Rule)

$$
F_k = \frac{\Delta x}{3}\left[f(x_k)+4 \cdot f(x_{k+1}) + f(x_{k+2})\right]
$$

In [ ]:
def num_int_2(f, xi, xe, n):
    if n % 2:
        n += 1

    x_array = py.linspace(xi, xe, n+1)
    delta_x = x_array[1] - x_array[0]
    delta_x_third = delta_x / 3.0

    integration_result = 0.0
    y0 = f(x_array[0])

    for i in range(1, n, 2):
        y1 = f(x_array[i])
        y2 = f(x_array[i+1])
        integration_result += delta_x_third * (y0 + 4*y1 + y2)
        y0 = y2

    return integration_result

## 매끄러운 함수에서의 수렴 차수<br>Convergence Order for a Smooth Function

먼저 $e^x$를 $0 \le x \le 1$ 구간에서 적분해 보자. 이 함수는 도함수가 모든 곳에서 존재하므로 이론적인 수렴 차수를 잘 보여준다.<br>
First, let's integrate $e^x$ over $0 \le x \le 1$. Since all derivatives of this function exist everywhere, it clearly shows the theoretical convergence order.

$$
\int_0^1 e^x dx = e - 1
$$

In [ ]:
exact_exp = py.exp(1) - 1.0
print('exact =', exact_exp)

구간 수 $n$을 바꿔가며 세 가지 방법의 오차를 측정하자.<br>
Let's measure the error of the three methods while varying the number of intervals $n$.

In [ ]:
n_list = [4, 8, 16, 32, 64, 128, 256, 512]

error_0_exp = []
error_1_exp = []
error_2_exp = []
delta_x_list = []

for n in n_list:
    delta_x_list.append(get_delta_x(0, 1, n))
    error_0_exp.append(abs(num_int_0(np.exp, 0, 1, n) - exact_exp))
    error_1_exp.append(abs(num_int_1(np.exp, 0, 1, n) - exact_exp))
    error_2_exp.append(abs(num_int_2(np.exp, 0, 1, n) - exact_exp))

In [ ]:
py.loglog(delta_x_list, error_0_exp, '.-', label='0th (Rectangle)')
py.loglog(delta_x_list, error_1_exp, '.-', label='1st (Trapezoid)')
py.loglog(delta_x_list, error_2_exp, '.-', label='2nd (Simpson)')

# 기울기 참고선
# Reference slope lines
dx = np.array(delta_x_list)
py.loglog(dx, dx * 1e0, 'k--', alpha=0.3, label='slope 1')
py.loglog(dx, dx**2 * 1e1, 'k-.', alpha=0.3, label='slope 2')
py.loglog(dx, dx**4 * 1e3, 'k:', alpha=0.3, label='slope 4')

py.xlabel('$\\Delta x$')
py.ylabel('$|E|$')
py.title('$\\int_0^1 e^x dx$ : Convergence Order\n$\\int_0^1 e^x dx$ : 수렴 차수')
py.legend(loc=0)
py.grid(True)
py.show()

log-log 그래프에서 각 방법의 기울기가 이론적 수렴 차수와 일치하는 것을 확인할 수 있다.<br>
On the log-log plot, we can confirm that the slope of each method matches the theoretical convergence order.

## 기울기 수치 확인<br>Numerical Verification of the Slopes

인접한 두 점 사이의 기울기를 계산하면 수렴 차수를 수치적으로 확인할 수 있다.<br>
By computing the slope between two adjacent points, we can numerically verify the convergence order.

$$
p \approx \frac{\log(E_2 / E_1)}{\log(\Delta x_2 / \Delta x_1)}
$$

In [ ]:
def compute_slopes(delta_x_list, error_list):
    slopes = []
    for i in range(len(error_list) - 1):
        if error_list[i] > 0 and error_list[i+1] > 0:
            slope = (np.log(error_list[i+1]) - np.log(error_list[i])) / \
                    (np.log(delta_x_list[i+1]) - np.log(delta_x_list[i]))
            slopes.append(slope)
        else:
            slopes.append(float('nan'))
    return slopes

In [ ]:
slopes_0 = compute_slopes(delta_x_list, error_0_exp)
slopes_1 = compute_slopes(delta_x_list, error_1_exp)
slopes_2 = compute_slopes(delta_x_list, error_2_exp)

print('0th order slopes:', [f'{s:.2f}' for s in slopes_0])
print('1st order slopes:', [f'{s:.2f}' for s in slopes_1])
print('2nd order slopes:', [f'{s:.2f}' for s in slopes_2])

0차 적분의 기울기는 약 1, 1차 적분의 기울기는 약 2, 2차 적분의 기울기는 약 4에 가까운 것을 확인할 수 있다.<br>
We can confirm that the slope of 0th order is approximately 1, 1st order is approximately 2, and 2nd order is approximately 4.

## 반원에서의 수렴 차수<br>Convergence Order for the Half Circle

면적이 1인 반원에 같은 비교를 적용해 보자.<br>
Let's apply the same comparison to a half circle with area 1.

In [ ]:
import plot_num_int as pi

In [ ]:
r = pi.radius_of_half_circle_area(1)
exact_half_circle = 1.0

error_0_hc = []
error_1_hc = []
error_2_hc = []
delta_x_hc_list = []

for n in n_list:
    delta_x_hc_list.append(get_delta_x(-r, r, n))
    error_0_hc.append(abs(num_int_0(pi.half_circle, -r, r, n) - exact_half_circle))
    error_1_hc.append(abs(num_int_1(pi.half_circle, -r, r, n) - exact_half_circle))
    error_2_hc.append(abs(num_int_2(pi.half_circle, -r, r, n) - exact_half_circle))

In [ ]:
py.loglog(delta_x_hc_list, error_0_hc, '.-', label='0th (Rectangle)')
py.loglog(delta_x_hc_list, error_1_hc, '.-', label='1st (Trapezoid)')
py.loglog(delta_x_hc_list, error_2_hc, '.-', label='2nd (Simpson)')

# 기울기 참고선
# Reference slope lines
dx = np.array(delta_x_hc_list)
py.loglog(dx, dx * 5e-1, 'k--', alpha=0.3, label='slope 1')
py.loglog(dx, dx**2 * 5e-1, 'k-.', alpha=0.3, label='slope 2')

py.xlabel('$\\Delta x$')
py.ylabel('$|E|$')
py.title('Half circle (area=1) : Convergence Order\n반원 (면적=1) : 수렴 차수')
py.legend(loc=0)
py.grid(True)
py.show()

In [ ]:
slopes_0_hc = compute_slopes(delta_x_hc_list, error_0_hc)
slopes_1_hc = compute_slopes(delta_x_hc_list, error_1_hc)
slopes_2_hc = compute_slopes(delta_x_hc_list, error_2_hc)

print('0th order slopes:', [f'{s:.2f}' for s in slopes_0_hc])
print('1st order slopes:', [f'{s:.2f}' for s in slopes_1_hc])
print('2nd order slopes:', [f'{s:.2f}' for s in slopes_2_hc])

반원 함수는 양 끝점 $x = \pm r$ 에서 도함수가 발산하므로, 매끄러운 함수에서와 수렴 양상이 다를 수 있다.<br>
Since the half circle function has diverging derivatives at the endpoints $x = \pm r$, the convergence behavior may differ from that of a smooth function.

## 두 경우 비교<br>Comparing the Two Cases

In [ ]:
fig, axes = py.subplots(1, 2, figsize=(12, 5))

# exp(x)
ax = axes[0]
ax.loglog(delta_x_list, error_0_exp, '.-', label='0th (Rectangle)')
ax.loglog(delta_x_list, error_1_exp, '.-', label='1st (Trapezoid)')
ax.loglog(delta_x_list, error_2_exp, '.-', label='2nd (Simpson)')
dx = np.array(delta_x_list)
ax.loglog(dx, dx * 1e0, 'k--', alpha=0.3, label='slope 1')
ax.loglog(dx, dx**2 * 1e1, 'k-.', alpha=0.3, label='slope 2')
ax.loglog(dx, dx**4 * 1e3, 'k:', alpha=0.3, label='slope 4')
ax.set_xlabel('$\\Delta x$')
ax.set_ylabel('$|E|$')
ax.set_title('$\\int_0^1 e^x dx$')
ax.legend(loc=0, fontsize='small')
ax.grid(True)

# half circle
ax = axes[1]
ax.loglog(delta_x_hc_list, error_0_hc, '.-', label='0th (Rectangle)')
ax.loglog(delta_x_hc_list, error_1_hc, '.-', label='1st (Trapezoid)')
ax.loglog(delta_x_hc_list, error_2_hc, '.-', label='2nd (Simpson)')
dx = np.array(delta_x_hc_list)
ax.loglog(dx, dx * 5e-1, 'k--', alpha=0.3, label='slope 1')
ax.loglog(dx, dx**2 * 5e-1, 'k-.', alpha=0.3, label='slope 2')
ax.set_xlabel('$\\Delta x$')
ax.set_ylabel('$|E|$')
ax.set_title('Half circle (area=1)\n반원 (면적=1)')
ax.legend(loc=0, fontsize='small')
ax.grid(True)

py.tight_layout()
py.show()

매끄러운 함수 ($e^x$) 에서는 이론적인 수렴 차수가 잘 나타난다. 반면 반원처럼 끝점에서 도함수가 특이한 함수에서는 수렴 차수가 낮아질 수 있다.<br>
For a smooth function ($e^x$), the theoretical convergence order is clearly observed. On the other hand, for functions with derivative singularities at the endpoints, like the half circle, the convergence order may be reduced.

## 연습 문제<br>Exercises

Try this 1: Plot the convergence order of the three methods for $\int_0^{\pi} \sin(x) dx = 2$. Is the result closer to $e^x$ or to the half circle?<br>
도전 과제 1: $\int_0^{\pi} \sin(x) dx = 2$ 에 대해 세 가지 방법의 수렴 차수를 그려 보시오. 결과가 $e^x$의 경우와 반원의 경우 중 어디에 더 가까운가?

Try this 2: Plot the convergence order for $\int_0^1 \sqrt{x} \ dx = \frac{2}{3}$. How does the derivative singularity at $x=0$ affect the convergence?<br>
도전 과제 2: $\int_0^1 \sqrt{x} \ dx = \frac{2}{3}$ 에 대해 수렴 차수를 그려 보시오. $x=0$에서의 도함수 특이성이 수렴에 어떤 영향을 미치는가?

## 시험<br>Test

아래는 함수가 맞게 작동하는지 확인함<br>
Following cells verify whether the functions work correctly.

In [ ]:
import math

# exp(x) 수렴 차수 확인
# Verify convergence order for exp(x)
assert all(0.8 < s < 1.2 for s in slopes_0), f"0th order slopes for exp(x): {slopes_0}"
assert all(1.8 < s < 2.2 for s in slopes_1), f"1st order slopes for exp(x): {slopes_1}"
assert all(3.8 < s < 4.2 for s in slopes_2), f"2nd order slopes for exp(x): {slopes_2}"

In [ ]:
# 적분 함수가 올바르게 작동하는지 확인
# Verify that the integration functions work correctly
assert math.isclose(num_int_0(np.exp, 0, 1, 1000), exact_exp, rel_tol=1e-3), num_int_0(np.exp, 0, 1, 1000)
assert math.isclose(num_int_1(np.exp, 0, 1, 100), exact_exp, rel_tol=1e-4), num_int_1(np.exp, 0, 1, 100)
assert math.isclose(num_int_2(np.exp, 0, 1, 10), exact_exp, rel_tol=1e-6), num_int_2(np.exp, 0, 1, 10)

## Final Bell<br>마지막 종

In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");